### Оглавление

1. [Подготовка данных](#Подготовка-данных)
2. [Пропуски в данных](#Пропуски-в-данных)
3. [Кодирование категориальных признаков](#Кодирование-категориальных-признаков)
4. [Кодирование даты и времени](#Кодирование-даты-и-времени)

# Подготовка данных

Предлагается рассмотреть корпус [Bank Marketing](https://archive.ics.uci.edu/ml/datasets/Bank+Marketing).

Корпус содержит результаты обзвонов клиентов банка. Требуется предсказать открыл ли клиент вклад.

### Загрузка корпуса

Корпус автоматически скачается. Обратите внимание на аргументы `pd.read_csv()`.

In [ ]:
import os
import tempfile
import urllib
import zipfile
from contextlib import contextmanager

import numpy as np
import pandas as pd


DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip"
TARGET = "y"

@contextmanager
def download(url):
    with tempfile.TemporaryDirectory() as root:
        os.path.basename(url)
        filename = os.path.basename(urllib.parse.urlparse(url).path)
        path = os.path.join(root, filename)
        urllib.request.urlretrieve(url, path)
        yield path


@contextmanager
def unzip(filename):
    with tempfile.TemporaryDirectory() as root:
        with zipfile.ZipFile(filename, "r") as zfp:
            zfp.extractall(root)
        yield root

        
def parse_names(filename):
    names = []
    with open(filename) as fp:
        for line in fp:
            if not line.startswith("@attribute"):
                continue
            names.append(line.split()[1])
    return names


with download(DATA_URL) as data_path:
    with unzip(data_path) as data_root:
        data = pd.read_csv(os.path.join(data_root, "bank-full.csv"),
                           sep=";",
                           na_values=["unknown"])  # Пропуски в .csv файле помечены словом "unknown".
        
        
# Усложним задачу: добавим пропуски в числовой признак.
np.random.seed(0)
mask = np.random.rand(len(data)) < 0.7
data["balance"].where(mask, other=np.NaN, inplace=True)

### Exploratory data analysis (EDA)

In [ ]:
data.info()

In [ ]:
data.head()

Каждая из 45211 строк таблицы соотвествует одному звонку. В корпусе 16 признаков. Некоторые могут быть пропущены. Нужно предсказать поведение клиента (значение поля `y`).

### Train / test split

Выделим тренировочную и тестовую части корпуса. Также отделим признаки от целевого значения.

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, test_size=0.2, random_state=0)

X_train = data_train.drop([TARGET], axis=1)
X_test = data_test.drop([TARGET], axis=1)
y_train = data_train[TARGET]
y_test = data_test[TARGET]

print("Train size", len(X_train))
print("Test size", len(X_test))

# Пропуски в данных

Как мы увидели, часто значение признаков равно `NaN`. Это значит, что оно пропущено в исходном корпусе.

In [ ]:
data.isna().any().sum()  # Сколько признаков содержат неизвестные значения.

In [ ]:
# Увеличим число отображаемых строк.
try:
    pd.set_option("display.height", 30)  # Старые версии pandas.
except Exception:
    pd.set_option("display.max_rows", 30)  # Новые версии.
nan_count = data.isna().sum()  # Сколько неизвестных значений содержит каждый признак.
nan_count[nan_count > 0]

Большинство ML моделей не поддерживают пропуски.

**Задание 1.** Предлагается заполнить пропуски в числовых полях `X_train` и `X_test` средним значением из `X_train`.

*Подсказка*. Можно воспользоваться функциями `<df>.mean()` и `<df\>.fillna()`. Обратите внимание на аргумент `numeric_only` функции `mean`.

In [ ]:
#
# Ваш код.
#

assert abs(X_train_new.loc[9894, "balance"] - 1362.268770) < 1e-6
assert abs(X_test_new.loc[13318, "balance"] - 1362.268770) < 1e-6
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

**Задание 2.** В корпусе остались пропущенные признаки. Pandas не смог вычислить среднее для категориальных признаков. Заполните их наиболее вероятным значением.

*Подсказка*. Можно воспользоваться функциями `<df>.mode()` и `<df\>.fillna()`. Функция `mode` возвращает таблицу с несколькими строками на случай, если мода не одна. Нужно взять первую строку.

In [ ]:
#
# Ваш код.
#

assert X_train_new.loc[9894, "job"] == "blue-collar"
assert X_test_new.loc[18883, "job"] == "blue-collar"
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

Проверим, что пропусков не осталось.

In [ ]:
assert X_train.isna().any().sum() == 0
assert X_test.isna().any().sum() == 0

# Кодирование категориальных признаков

###  Как понять, что признак категориальный?

Идея - категориальные признаки те, в которых значения - строки или число уникальных значений достаточно мало.

In [ ]:
def find_cat(data, num_uniq=10):
    columns = []
    for name in data.columns:
        message = name
        if type(data[name][0]) == str:
            message += " строка,"
        if data[name].nunique() <= num_uniq:
            message += " мало уникальных"
        if message != name:
            columns.append(name)
            print(message)
    return columns
            
find_cat(data)

**Задание 3.** Реализуйте one-hot кодирование категориальных признаков, перечисленных в `CAT_COLUMNS`.

*Подсказка*. Можно воспользоваться функциями `pd.concat()` и `pd.get_dummies(<df>)`.

In [ ]:
CAT_COLUMNS = ["job", "marital", "education", "contact", "poutcome"]

#
# Ваш код.
#

assert X_train_new.loc[13932, "job_admin."] == 1
assert X_test_new.loc[14001, "education_tertiary"] == 1
print("Check OK!")
X_train = X_train_new
X_test = X_test_new
X_train.head()

Покажем, как можно сгруппировать числовые признаков по категориальным.

Например, для каждого значения признака `education` из нашей таблички можно посчитать среднее значение `balance`.

In [ ]:
def group_mean(data, cat_feature, real_feature):
    return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

group_mean(data, "education", "balance")

**Задание 4.** Замените `yes` и `no` в столбцах `YN_COLUMNS` на 1 и 0 соотвественно.

*Подсказка.* Можно воспользоваться методами `<df>[<column>].map()` и `<df>.drop([<column>], axis=1)`.

In [ ]:
YN_COLUMNS = ["default", "housing", "loan"]
MAPPING = {"yes": 1, "no": 0}

#
# Ваш код
#

assert X_train_new.loc[39946, "housing"] == 1
assert X_test_new.loc[32046, "default"] == 0
print("Check OK!")

X_train = X_train_new
X_test = X_test_new
X_train.head()

# Кодирование даты и времени

**Задание 5.** Реализуйте периодическое кодирование месяца (в новые поля `month_cos` и `month_sin`):

$cos(2 * \pi * \frac{x}{period})$, $sin(2 * \pi * \frac{x}{period})$

*Подсказка.* Можно воспользоваться методами `<df>[<column>].map()` и `<df>.drop([<column>], axis=1)`.

In [ ]:
ORDER = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
MONTH_MAPPING = {month: i for i, month in enumerate(ORDER)}

#
# Ваш код
#

assert abs(X_train_new.loc[13932, "month_cos"] + 1) < 1e-6
assert abs(X_test_new.loc[32046, "month_sin"] - 1) < 1e-6
print("Check OK!")

X_train = X_train_new
X_test = X_test_new
X_train.head()

Выходные значения тоже нужно преобразовать в 0,1

In [ ]:
y_train = y_train.map(MAPPING)
y_test = y_test.map(MAPPING)
y_train.head()

# Проверим, что теперь все данные числовые

In [ ]:
X_train.info()